In [144]:
from torch import Tensor
import torch as torch
import time as time

### 1 Multiple views of a storage

In [145]:
def multiple_views_storage():
    x = torch.full((13, 13), 1)
    x.narrow(0, 1, 1).fill_(2) 
    x.narrow(0, 6, 1).fill_(2)
    x.narrow(0, 11, 1).fill_(2)
    x.narrow(1, 1, 1).fill_(2)
    x.narrow(1, 6, 1).fill_(2)
    x.narrow(1, 11, 1).fill_(2)
    z = x.narrow(0, 3, 2).narrow(1, 3, 2)
    z.fill_(3)
    z = x.narrow(0, 8, 2).narrow(1, 3, 2)
    z.fill_(3)
    z = x.narrow(0, 3, 2).narrow(1, 8, 2)
    z.fill_(3)
    z = x.narrow(0, 8, 2).narrow(1, 8, 2)
    z.fill_(3)
    return x

In [146]:
multiple_views_storage()
# x.narrow(dim, index, size)
# axis = 1 means dimension 1 : columns
# axis = 0 means dimension 0 : rows

tensor([[1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1],
        [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
        [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1],
        [1, 2, 1, 3, 3, 1, 2, 1, 3, 3, 1, 2, 1],
        [1, 2, 1, 3, 3, 1, 2, 1, 3, 3, 1, 2, 1],
        [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1],
        [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
        [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1],
        [1, 2, 1, 3, 3, 1, 2, 1, 3, 3, 1, 2, 1],
        [1, 2, 1, 3, 3, 1, 2, 1, 3, 3, 1, 2, 1],
        [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1],
        [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
        [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1]])

In [147]:
x = torch.full((13, 13), 1)
x.narrow(1, 6, 1).fill_(2)

tensor([[2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [2]])

### 2 Eigendecomposition

In [165]:
M = Tensor(20,20).normal_()
M = Tensor.float(M)

D = torch.diag(torch.arange(1, 21))
D = Tensor.float(D)

M_inv = Tensor.inverse(M)

A = torch.mm(M_inv, torch.mm(D, M))

Tensor.eig(A)

torch.return_types.eig(
eigenvalues=tensor([[ 1.0000,  0.0000],
        [ 2.0000,  0.0000],
        [20.0000,  0.0000],
        [ 3.0000,  0.0000],
        [ 4.0000,  0.0000],
        [ 5.0000,  0.0000],
        [19.0000,  0.0000],
        [18.0000,  0.0000],
        [ 6.0000,  0.0000],
        [ 7.0000,  0.0000],
        [17.0000,  0.0000],
        [ 8.0000,  0.0000],
        [16.0000,  0.0000],
        [ 9.0000,  0.0000],
        [10.0000,  0.0000],
        [15.0000,  0.0000],
        [11.0000,  0.0000],
        [12.0000,  0.0000],
        [14.0000,  0.0000],
        [13.0000,  0.0000]]),
eigenvectors=tensor([]))

## 3 Flops per second

In [166]:
A = Tensor(5000, 5000).normal_() # First square matrix of dimension 5000 × 5000

B = Tensor(5000, 5000).normal_() # Seconde square matrix of dimension 5000 × 5000

x_0 = time.perf_counter()

torch.mm(A, B) # Product of A and B

x_1 = time.perf_counter()

time_estimation = 5000*5000*5000 / (x_1 - x_0) # matrix multiplication is approx. O(n^3)

time_estimation

66301151072.84794

## 4 Playing with strides

In [167]:
def mul_row(x):
    start_time = time.perf_counter()
    nrows = (x.size())[0]
    for i in range(nrows):
        y = x.narrow(0, i, 1)
        y *= (i+1)
    end_time = time.perf_counter()
    return end_time - start_time

def mul_row_fast(x):
    start_time = time.perf_counter()
    nrows = (x.size())[0]
    y = torch.arange(1, nrows+1).expand(x.size()[1], nrows).t()
    end_time = time.perf_counter()
    return end_time - start_time

def part_four():
    m = Tensor(10000, 400).fill_(2.0)
    t1 = mul_row(m)
    t2 = mul_row_fast(m)
    return t1 / t2

In [168]:
print("Part four time(mul_row) / time(mul_row_fast) : {}".format(part_four()))

Part four time(mul_row) / time(mul_row_fast) : 1592.1525473283482
